In [1]:
import math
import numpy as np
import pandas as pd
from plotly.graph_objs import *
import plotly
import plotly.graph_objs as go
import pickle

dtor = math.pi/180

# Load Map + make contours 
# (optional, only if you want to recreate contours)
# (otherwise, skip down to loading contours from file

In [2]:
from astropy.utils.data import download_file
from ligo.skymap.io import *
import ligo.skymap

In [3]:
url= 'https://gracedb.ligo.org/apiweb/superevents/S190510g/files/LALInference.fits.gz,0'
skymapfile = download_file(url, cache=True) 

In [4]:
skymap, metadata = ligo.skymap.io.fits.read_sky_map(url, nest=None)

In [ ]:
#VERY VERY VERY SLOW. Only run once to generate the file. There is better way to do this. 
#Writes to .geojson format file.
#from ligo.skymap.tool.ligo_skymap_contour import main
#main([skymapfile,'--contour','50','90','--simplify', '-o', 'contours-smooth.json']) #add --simplify

# Load GW contours from geojson file

In [7]:
#smoothed caled contours.json in home directory
contours_data=pd.read_json('contours-smooth.json')

In [8]:
pathlist = []
for contour in contours_data['features']:
    level = contour['properties']['credible_level']
    pathlist.extend(contour['geometry']['coordinates'])

In [9]:
def make_contours(pathlist):
    footprints=[]
    for path in pathlist:
        ras, decs = zip(*path)
        ras=list(ras)
        decs=list(decs)
        #ras.reverse()
        #decs.reverse()
        footprint = Scattergeo(
            lon=ras,
            lat=decs,
            line=scattergeo.Line(
                color='rgb(127, 127, 127)',
                width=.5,
            ),
            visible= True,
            mode='lines',
            hoverinfo='none')
        footprints.append(footprint)
    return footprints

In [73]:
GWfootprints= make_contours(pathlist)

# Polygons to Footprints

In [67]:
#this takes a polygon in skycoords in the form of a list of tuples of ra,decs ie ((0,0)(0,1)(1,1)(1,0))
#and returns a footprint object
def polygon2footprint(polygon, instrument):
    #split list of poly corners into ra list and dec list
    ras,decs = zip(*polygon)
    ras=list(ras)
    decs=list(decs)
    #have to 'close' the poly list for the filling to work correctly
    ras.append(ras[0])
    decs.append(decs[0])
    # if you make the plot and the whole damn thing is filled in, reverse the direction of the footprints
    # plotly seems to determine the 'inside'  and 'outside' of footprints based on the clockwise/counter direction of the trace
    #ras.reverse()
    #decs.reverse()
    if instrument == 'XRT':
        color = 'rgb(0, 139, 205)'
    elif instrument == 'UVOT':
        color = 'rgb(213, 62, 79)'
    
    footprint = Scattergeo(
            lon=ras,
            lat=decs,
            line=scattergeo.Line(
                color=color,
                width=.5,
            ),
            fill="toself",
            mode='lines',
            hoverinfo='none')
    return footprint

In [23]:
with open('xrtpolys', 'rb') as f:
    XRTpolygons = pickle.load(f)

In [63]:
with open('uvotpolys', 'rb') as f:
    UVOTpolygons = pickle.load(f)

In [68]:
UVOTfootprints = []
for polygon in UVOTpolygons:
    footprint = polygon2footprint(polygon,'UVOT')
    UVOTfootprints.append(footprint)

In [69]:
XRTfootprints = []
for polygon in XRTpolygons:
    footprint = polygon2footprint(polygon,'XRT')
    XRTfootprints.append(footprint)

# Make the Plot

In [75]:
data = [
        dict(
            lon=list(range(0, 360, 60))+[180]*4,
            lat=[0]*6+[-60, -30, 30, 60],
            text=list(range(0, 360, 60))+[-60, -30, 30, 60],
            hoverinfo='none',
            mode='text',
            type='scattergeo'
        )
]
data.extend(GWfootprints)
data.extend(XRTfootprints)
data.extend(UVOTfootprints)

#data.extend([BATtrace1])
#data.extend([BATtrace50])


#this is how i make the buttons to turn different Instruments on and off
baselist = [True]+([True]*len(GW_footprints))
xrtshowlist = baselist + ([True]*len(XRTfootprints)) + ([False]*len(UVOTfootprints))
uvotshowlist= baselist + ([False]*len(XRTfootprints)) + ([True]*len(UVOTfootprints))

updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = 'UVOT',
                 method = 'update',
                 args = [{'visible': uvotshowlist},
                         {'title': 'UVOT Fields'}]),
            dict(label = 'XRT',
                 method = 'update',
                 args = [{'visible': xrtshowlist},
                         {'title': 'XRT Fields'}])
        ]),
    )
])

layout = {
    'title': 'GW Tiling Plot',
    'hovermode': 'closest',
    'showlegend': False,
    'updatemenus': updatemenus,
    'geo': {
        'projection': {
            'type': 'mollweide',
        },
        'showcoastlines': False,
        'lonaxis': {
            'showgrid': True,
            'range': [0, 360],
        },
        'lataxis': {
            'showgrid': True,
            'range': [-90, 90],
        },
    }
}
print('This takes <20 sec to run, be patient.')
plotly.offline.plot(go.Figure(data=data, layout=layout)) 
#right now this runs and makes a temp-plot.html file in your directory, 
#but you have to stop the kernel manually for some reason...
#better to write the div explicitly as below, but I usually use this method for quick testing

This takes <20 sec to run, be patient.


'temp-plot.html'

# Make webpage

In [ ]:
#create the div only, for inclusion in a webpage, need to source <script src="https://cdn.plot.ly/plotly-latest.min.js"></script> 
#before div
thediv = plotly.offline.plot(go.Figure(data=data, layout=layout), include_plotlyjs=False, output_type='div')

In [ ]:
#something like this
html_string = '''
<html>
    <head>
      <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
      <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
      <style>body{ margin:0 100; background:whitesmoke; }</style>
    </head>
    <body>
      <h1>Monthly Report</h1>
      ''' + thediv + '''
    </body>
</html>'''

In [ ]:
#write HTML file
with open("example.html", 'w') as f:
    f.write(html_string)